In [ ]:
#select the files

timeframe='2019a'
inputpath='/home/davoud/Documents/cs project/comment source/edited data oct/'
input_here= inputpath+timeframe+'.csv'
pathldavis = '/home/davoud/Documents/cs project/trial/October 22/' + timeframe + '.html'
pathshortsumary1= '/home/davoud/Documents/cs project/trial/October 22/' +  timeframe  + '_all_comments'+ '.csv'
pathshortsumary2= '/home/davoud/Documents/cs project/trial/October 22/' + timeframe  +'_summary_table'+  '.csv'
pathshortsumary3= '/home/davoud/Documents/cs project/trial/October 22/' + timeframe  +'_sample_text'+  '.csv'


In [ ]:
# importing libraries 
import re
import numpy as np
import pandas as pd
from pprint import pprint
import csv
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
#  stop-words list
import stop_words
stopwords = set(stop_words.get_stop_words('en'))
stopwords=list(stopwords)
from nltk.stem import WordNetLemmatizer

#add additional item to stopwords
stopwords=list(stopwords)
stop_words=stopwords
stop_words.extend(['from', 'subject', 're', 'edu', 'use','bitcoin','bitcoins','would','com', 'quote', 'pmquote', 'amquote', 'just', 'don', 'one', 'thing', 'even', 'way', 'maybe', 'also', 'please', 'well', 'actually', 'something','going', 'anything', 'le', 'ever', 'say', 'see', 'likely', 'per', 'another', 'someone', 'let', 'anyone', 'doesn', 'include', 'doe', 'exactly', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'like', 'said', 'guy', 'will', 'can', 'able', 'people', 'become', 'tell', 'hey', 'much', 'many', 'lol', 'lot', 'want', 'still', 'really', 'think', 'didn', 'isn', 'post', 'edited', 'could','http_www', 'use', 'get', 'ask', 'say', 'self', 'look',  'wonder', 'sound', 'make' ,'come' ,'do', 'go', 'probably' ,'need' ,'next' ,'day' , 'today' ,'tomorrow', 'yesterday' ,'month', 'week' ,'year' ,'total' ,'however' ,'similar' ,'half' ,'read' ,'forum' ,'room', 'enough', 'put', 'http' ,'www' , 'btc', 'call', 'cool',  'search', 'say' ,'talk' ,'name', 'somewhere', 'indeed', 'use','give', 'keep', 'user','member','go','often', 'unit','good', 'seem','completely', 'different', 'suggestion', 'complete', 'mention','small','big','find','sound','wish','Now', 'day','month','week','year','guy','one','sound','bad','old','sure','give','main','sense','open','close','use','tool','wrong','say','test','keep','lose','soon','pass','directly','send','receive','case','see','true','false','concern','must','should','could','obviously','stand','seemingly','present','high','low','excellent','seriously','now','know','take','Now', 'day','month','week','year','guy','one','sound','bad','old','sure','give','main','sense','open','close','use','tool','wrong','say','test','keep','lose','soon','pass','directly','send','receive','case','see','true','false','concern','must','should','could','obviously','stand','seemingly','present','high','low','excellent','seriously','now','know','take','try'])


In [ ]:
#feeding  data source

df = pd.read_csv(input_here)
df.head()

In [ ]:

# Data Cleaning
# Convert to list
data = df.for_lda.values.tolist()

print ('Initial Number of Comments = ',len(data) )

# Remove Emails
# Remove new line characters
# Remove distracting single quotes
k=0
temp100=[]
for sent in data:
    try: 
        sent=str(sent)
        sent= sent.lower()
        sent = re.sub(r'http\S+', ' ', sent) # Remove websites
        sent=  re.sub(r'\S+.(com|org)\S+', '', sent) # Remove website2
        
        sent = re.sub('\S*@\S*\s?', '', sent) # Remove Emails
        sent = re.sub('\s+', ' ', sent) # Remove new line characters
        sent = re.sub("\'", "", sent) # Remove distracting single quotes
        sent= re.sub( "[^a-zA-Z]", " ", sent)
        if len(sent.split())>5 :
            temp100.append(sent)
            k +=1

    except TypeError:
        continue 
data= temp100
print ('Remaing Comment after pre Cleaning = ',k)



In [ ]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ'])



for item in data_lemmatized:
    for element in item:
        if len(element)<3:
            item.remove(element)



# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

 # Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/home/davoud/Documents/malletm/mallet-2.0.8/bin/mallet'  # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=id2word)



In [ ]:
  # Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word,
                                               coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


In [ ]:
mallet_lda_model= gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(mallet_lda_model, corpus, id2word)
vis
# save graph
pyLDAvis.save_html(vis, pathldavis)

In [ ]:
# show list of topics
optimal_model=ldamallet
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))
df_topic_per = pd.DataFrame(optimal_model.print_topics(num_words=10),columns=['Topic Number','Keywords Perc'])

In [ ]:

def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)
# save the list
df_dominant_topic.to_csv(pathshortsumary1)
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(15)

sent_topics_sorteddf_mallet.to_csv(pathshortsumary3)

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

#sent_topics_sorteddf_mallet


df_dominant_123 = pd.concat([ topic_counts, topic_contribution,df_topic_per], axis=1)
df_dominant_123.columns=['Count_of_Comments', 'Perc_Contrib','Topic_Num', 'Topic_Keywords']
df_dominant_123.to_csv(pathshortsumary2)
df_dominant_123